In [ ]:
MASTERNODE_PRIVATE_IP="192.168.2.187"

In [ ]:
from pyspark.sql import SparkSession

# New API
spark_session = SparkSession.builder\
        .master(f"spark://{MASTERNODE_PRIVATE_IP}:7077") \
        .appName("Lecture2_Example1_Shared_State")\
        .config("spark.executor.cores",4)\
        .config("spark.dynamicAllocation.enabled", True)\
        .config("spark.dynamicAllocation.shuffleTracking.enabled", True)\
        .config("spark.shuffle.service.enabled", False)\
        .config("spark.dynamicAllocation.executorIdleTimeout","30s")\
        .config("spark.driver.port",9999)\
        .config("spark.blockManager.port",10005)\
        .getOrCreate()


# Old API (RDD)
spark_context = spark_session.sparkContext
spark_context.setLogLevel("ERROR")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/02/09 16:57:37 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


---
## 1. Broadcast Variables

In [2]:
# Immutable data for all workers
broadcast_variable = spark_context.broadcast({"apples": 100, "oranges": 42, "pears": 1})

In [3]:
rdd = spark_context.parallelize(["apples", "pears"])

# Convert fruit to amounts of fruit.
# "value" is captured in the closure, but its only a handle - the dictionary is serialized via a different route
result = rdd.map(lambda x: broadcast_variable.value[x])\
            .collect()

print(result)

[Stage 0:>                                                          (0 + 2) / 2]

[100, 1]


## 2. Accumulator Variables

In [4]:
# ACCUMULATOR 
accumulator_1 = spark_context.accumulator(0)
rdd2 = spark_context.parallelize([1,2,3,4,5,6], 2)

In [5]:
result = rdd2.foreach(lambda x: accumulator_1.add(1))

print(accumulator_1.value)

6


In [6]:
# release the cores for another application!
spark_context.stop()